# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --type json -d uk_food -c establishments --drop --jsonArray .\Resources\establishments.json`

In [9]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [10]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [11]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'config', 'local', 'met', 'travel_db', 'uk_food']


In [12]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [14]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [15]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'The Coastguard Inn',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 1034540,
 'LocalAuthorityBusinessID': 'PI/000078691',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-08-17T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65e4fb5de47be9158bd23a7a'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1034540',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T0

In [16]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

In [18]:
establishments.count_documents({})



39779

In [19]:
query = {'BusinessName': "Penang Flavours"}
# Print the number of results
print("Number of documents in result:", establishments.count_documents(query))


Number of documents in result: 0


1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [20]:
# Create a dictionary for the new restaurant data

# Define the document for the new restaurant
new_restaurant = {
    'BusinessName': 'Penang Flavours',
    'BusinessType': 'Restaurant',
    'AddressLine1': '123 Challenge st',
    'AddressLine2': 'Greenwich',
    'BusinessTypeID': None, # Assuming we don't have a specific type ID
    'RatingValue': 'AwaitingInspection', # Since it hasn't been rated yet
    'LocalAuthorityName': 'Greenwich',
    'PostCode': 'Unknown', # Assuming postcode is not provided
    'RatingDate': None, # No rating date yet
    'NewRatingPending': True, # Indicating that a rating is pending
    # Add any additional fields as needed
}

# Insert the new restaurant into the 'establishments' collection
result = db.establishments.insert_one(new_restaurant)

# Print the ID of the new document
print(f"New restaurant added with ID: {result.inserted_id}")

New restaurant added with ID: 65e5149d7a36f82c7ba349a1


In [21]:
# Check that the new restaurant was inserted
query = {'BusinessName': "Penang Flavours"}
# Print the number of results
print("Number of documents in result:", establishments.count_documents(query))

Number of documents in result: 1


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [37]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields

# Define the query to find the BusinessTypeID for "Restaurant/Cafe/Canteen"
query = {'BusinessType': 'Restaurant/Cafe/Canteen'}

# Define the projection to return only BusinessTypeID and BusinessType
projection = {'_id': 0, 'BusinessTypeID': 1, 'BusinessType': 1}

# Execute the query and projection
result = db.establishments.find_one(query, projection)

# Print the result
if result:
    print(result)
else:
    print("No document found matching the criteria.")

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [25]:
# Update the new restaurant with the correct BusinessTypeID
# Define the filter to find the document for "Penang Flavours"
filter = {'BusinessName': 'Penang Flavours'}

# Define the update to be applied
update = {'$set': {'BusinessTypeID': 1,
                   'BusinessType': 'Restaurant/Cafe/Canteen'}}

# Execute the update operation
result = db.establishments.update_one(filter, update)

# Check if the document was updated and print a message
if result.matched_count:
    print("Restaurant updated successfully.")
else:
    print("Restaurant not found.")

Restaurant updated successfully.


In [28]:
# Confirm that the new restaurant was updated
# Define the filter to find the document for "Penang Flavours"
filter = {'BusinessName': 'Penang Flavours'}

# Retrieve the document
restaurant_document = db.establishments.find_one(filter)

# Check if the document was found and print it
if restaurant_document:
    print("Restaurant details:")
    for key, value in restaurant_document.items():
        print(f"{key}: {value}")
else:
    print("Restaurant not found.")

Restaurant details:
_id: 65e5149d7a36f82c7ba349a1
BusinessName: Penang Flavours
BusinessType: Restaurant/Cafe/Canteen
AddressLine1: 123 Challenge st
AddressLine2: Greenwich
BusinessTypeID: 1
RatingValue: AwaitingInspection
LocalAuthorityName: Greenwich
PostCode: Unknown
RatingDate: None
NewRatingPending: True


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [29]:
# Find how many documents have LocalAuthorityName as "Dover"
# Define the query to match documents where LocalAuthorityName is "Dover"
query = {'LocalAuthorityName': 'Dover'}

# Use count_documents to find how many documents match the query
count = db.establishments.count_documents(query)

print(f"Number of documents with LocalAuthorityName as 'Dover': {count}")

Number of documents with LocalAuthorityName as 'Dover': 994


In [30]:
# Delete all documents where LocalAuthorityName is "Dover"
# Define the deletion criteria
criteria = {'LocalAuthorityName': 'Dover'}

# Perform the deletion
delete_result = db.establishments.delete_many(criteria)

# Print how many documents were deleted
print(f"Documents deleted: {delete_result.deleted_count}")

Documents deleted: 994


In [31]:
# Check if any remaining documents include Dover
# Define the query to match documents where LocalAuthorityName is "Dover"
query = {'LocalAuthorityName': 'Dover'}

# Use count_documents to find how many documents match the query
count = db.establishments.count_documents(query)

print(f"Number of documents with LocalAuthorityName as 'Dover': {count}")

Number of documents with LocalAuthorityName as 'Dover': 0


In [32]:
# Check that other documents remain with 'find_one'
pprint(db.establishments.find_one())

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65e4fb5de47be9158bd23d60'),
 'geocode': {'latitude': '51.083812', 'longitude': '1.195625'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': 

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [33]:
# Change the data type from String to Decimal for longitude and latitude

# Define an aggregation pipeline for the update operation
# This converts the latitude and longitude fields from strings to Decimal128,
# which is MongoDB's decimal type. If your fields are already in a different numeric type
# or if you prefer a different type (like float), adjust accordingly.
pipeline = [
    {
        '$set': {
            'geocode.latitude': {
                '$convert': {
                    'input': '$geocode.latitude',
                    'to': 'decimal',
                    # Optional: Handle errors during conversion
                    'onError': 'Error',  
                    'onNull': 'Error'
                }
            },
            'geocode.longitude': {
                '$convert': {
                    'input': '$geocode.longitude',
                    'to': 'decimal',
                    # Optional: Handle errors during conversion
                    'onError': 'Error',
                    'onNull': 'Error'
                }
            }
        }
    }
]

# Execute the update operation
result = db.establishments.update_many({}, pipeline)

# Print how many documents were updated
print(f"Documents updated: {result.modified_count}")

Documents updated: 38786


Use `update_many` to convert `RatingValue` to integer numbers.

In [34]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4092, 'nModified': 4092, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [35]:
# Change the data type from String to Integer for RatingValue
# Define an aggregation pipeline for the update operation
# This converts the RatingValue field from a string to an integer.
pipeline = [
    {
        '$set': {
            'RatingValue': {
                '$convert': {
                    'input': '$RatingValue',
                    'to': 'int',
                    # Optional: Specify what to do if the conversion fails
                    'onError': 'Error',
                    'onNull': 'Error'
                }
            }
        }
    }
]

# Execute the update operation for documents where RatingValue is a numeric string
result = db.establishments.update_many(
    # This filter attempts to exclude non-numeric values by using a regex match.
    # It's a basic filter and may need adjustment to accurately match your data.
    {'RatingValue': {'$regex': '^\d+$'}},
    pipeline
)

# Print how many documents were updated
print(f"Documents updated: {result.modified_count}")

Documents updated: 34694


In [36]:
# Check that the coordinates and rating value are now numbers
# Retrieve a single document from the 'establishments' collection
sample_document = db.establishments.find_one()

# Print the sample document's latitude, longitude, and RatingValue, along with their types
if sample_document and 'geocode' in sample_document:
    latitude = sample_document['geocode']['latitude']
    longitude = sample_document['geocode']['longitude']
    print(f"Latitude: {latitude}, Type: {type(latitude)}")
    print(f"Longitude: {longitude}, Type: {type(longitude)}")
else:
    print("No geocode information available in the sample document.")

if 'RatingValue' in sample_document:
    rating_value = sample_document['RatingValue']
    print(f"RatingValue: {rating_value}, Type: {type(rating_value)}")
else:
    print("No RatingValue field available in the sample document.")

Latitude: 51.083812, Type: <class 'bson.decimal128.Decimal128'>
Longitude: 1.195625, Type: <class 'bson.decimal128.Decimal128'>
RatingValue: 5, Type: <class 'int'>
